In [3]:
from langchain_core.tools import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from typing import Dict, Any, List, Optional
from langchain.agents.middleware import AgentState, AgentMiddleware
from langchain.messages import HumanMessage, AIMessage, SystemMessage
from typing_extensions import TypedDict, NotRequired

C:\Users\dhanu\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def create_production_tools() -> List[Tool]:
    """Create production-ready tools with error handling."""
    
    # Search tool with error handling
    def safe_search(query: str) -> str:
        """Perform web search with error handling."""
        try:
            search = DuckDuckGoSearchRun()
            result = search.run(query)
            return result
        except Exception as e:
            logger.error(f"Search failed: {e}")
            return f"Search temporarily unavailable. Error: {str(e)}"
    
    search_tool = Tool(
        name="web_search",
        func=safe_search,
        description=(
            "Search the web for current information. "
            "Use this when you need up-to-date facts or recent events."
        )
    )
    
    # Calculator tool
    def safe_calculate(expression: str) -> str:
        """Safely evaluate mathematical expressions."""
        try:
            # Security: only allow safe characters
            allowed = set("0123456789+-*/.() ")
            if not all(c in allowed for c in expression):
                return "Error: Invalid characters in expression"
            
            result = eval(expression)
            return f"Result: {result}"
        except Exception as e:
            return f"Calculation error: {str(e)}"
    
    calc_tool = Tool(
        name="calculator",
        func=safe_calculate,
        description=(
            "Perform mathematical calculations. "
            "Input should be a valid mathematical expression like '2 + 2' or '(15 * 8) / 4'."
        )
    )
    
    return [search_tool, calc_tool]
